## Functional connectivity

This notebook used resting-state time-series to calculate the subject-level connectivity matrix of the Cognitive Control network as identified in [@dosenbach2007]. The connectivity matrix, of shape $N_{\text{subjects}} \times N_{\text{regions}} \times N_{\text{regions}}$, will be stored as an annotated NetCDF4 file in the following file: `data/julia2018_resting/connectivities.nc`.

## Setup

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import nilearn
from nilearn import connectome

Load the time-series of the cognitive control network:

In [4]:
DATASET = xr.open_dataset('data/julia2018_resting/timeseries_dosenbach2007.nc')

## Fit connectivity matrices


In [5]:
# some subjects are missing resting time-series, so we need to filter them out.

missing_mask = DATASET['timeseries'].isnull().all(dim=['region','timestep'])
missing_indices = np.flatnonzero(missing_mask)

valid_timeseries = [ts.T for ts in DATASET['timeseries'].values[~missing_mask]]

In [11]:
# outputs
conn_kinds = ['covariance','correlation', 'partial correlation', 'tangent', 'precision']

for kind in conn_kinds:
  
  # calculate connectivity measure
  cm = nilearn.connectome.ConnectivityMeasure(kind=kind, vectorize=False)
  conn = cm.fit_transform(valid_timeseries)
  
  # now refill the places of the missing time-series with nan values
  nan_insertion_indices = missing_indices - np.arange(missing_indices.shape[0])
  conn = np.insert(conn, nan_insertion_indices, np.full_like(conn[0], np.nan), axis=0)

  # DEBUG: just to make sure missing values are handled correctly
  assert np.equal(np.isnan(conn).all(axis=(1,2)), missing_mask).all()
  
  # add to the dataset
  db_key_name = kind.replace(' ', '_') + '_connectivity'
  DATASET[db_key_name] = xr.DataArray(conn, dims=['subject', 'region', 'region'])

# now store the dataset
DATASET.attrs['description'] = 'Connectivity matrices extracted from resting timeseries.'
DATASET.to_netcdf('data/julia2018_resting/connectivity_dosenbach2007.nc', engine='netcdf4')

DATASET

<xarray.Dataset>
Dimensions:                           (subject: 34, region: 39, timestep: 125,
                                       mni152_coord: 3)
Coordinates:
  * subject                           (subject) object 'AVGP01' ... 'NVGP19NEW'
  * region                            (region) object 'R IPS' ... 'vmPFC'
  * mni152_coord                      (mni152_coord) object 'x' 'y' 'z'
  * timestep                          (timestep) int64 0 1 2 3 ... 122 123 124
Data variables: (12/16)
    timeseries                        (subject, region, timestep) float64 -0....
    regions                           (region, mni152_coord) int64 30 -61 ... -2
    groups                            (subject) object 'AVGP' 'AVGP' ... 'NVGP'
    age                               (subject) float64 27.0 19.0 ... 19.0 22.0
    sex                               (subject) object 'M' 'M' 'M' ... 'M' 'M'
    group                             (subject) object 'AVGP' 'AVGP' ... 'NVGP'
    ...                                ...
    false_alarm_pct                   (subject) float64 3.13 0.0 ... 70.31 3.13
    covariance_connectivity           (subject, region, region) float64 1.0 ....
    correlation_connectivity          (subject, region, region) float64 1.0 ....
    partial_correlation_connectivity  (subject, region, region) float64 1.0 ....
    tangent_connectivity              (subject, region, region) float64 0.039...
    precision_connectivity            (subject, region, region) float64 3.633...
Attributes:
    description:  Connectivity matrices extracted from resting timeseries.